# Scraping and Sentiment Analyzer

## Imports

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [ ]:
!pip install -q news-please pygooglenews cchardet transformers torch vaderSentiment xlsxwriter gspread gspread-pandas oauth2client

     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 263 kB 11.6 MB/s 
     |████████████████████████████████| 3.5 MB 44.0 MB/s 
     |████████████████████████████████| 125 kB 60.4 MB/s 
     |████████████████████████████████| 149 kB 49.0 MB/s 
     |████████████████████████████████| 254 kB 47.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 3.0 MB 33.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 369 kB 36.6 MB/s 
     |████████████████████████████████| 211 kB 46.7 MB/s 
     |████████████████████████████████| 981 kB 38.7 MB/s 
     |████████████████████████████████| 47 kB 3.6 MB/s 
     |████████████████████████████████| 40 kB 1.3 MB/s 
     |████████████████████████████████| 3.8 MB 24.4 MB/s 
     |████████████████████████████████| 8.5 MB 11.2 MB/s 
     |████████████████████████████████| 547 kB 35.7 MB/s 
     |██████████████████

In [ ]:
import pandas
import numpy
from IPython.display import clear_output
from datetime import datetime, date, timedelta
from time import mktime
import math

from pygooglenews import GoogleNews
from newsplease import NewsPlease
import torch
import requests
from transformers import BertTokenizer, BertForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from  matplotlib.colors import LinearSegmentedColormap

import gspread
import gspread_pandas

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## News Scraping

### News Fetcher

In [ ]:

def get_news(google_news_code):
  gn = GoogleNews(lang = 'en', country = 'IN')
  company_news = gn.topic_headlines(google_news_code)
  entries = []
  rows = []
  # can change sources below
  sources = ['bloombergquint', 'business-standard', 'cnbctv18',
             'economictimes', 'financialexpress', 'indiainfoline',
             'livemint', 'moneycontrol', 'ndtv',
             'newindianexpress', 'news18', 'thehindubusinessline',
             'timesofindia', 'zeebiz']
  # change date range below 
  date_filter = pandas.to_datetime((date.today() - timedelta(days=2)),
                                   format='%Y-%m-%d')
 
  for entry in company_news['entries']:
    upload_date = (datetime.fromtimestamp(mktime(entry['published_parsed']))).strftime('%Y-%m-%d')
    upload_date = pandas.to_datetime(upload_date,format='%Y-%m-%d')
    if upload_date>=date_filter:
      #make the below "not in" and this will solve the paid article issue
      if any(source in entry['source']['href'] for source in sources):
        entries.append(entry)

  for entry in entries:
    try:
      link = requests.get(entry['link'])
      time_of_upload = datetime.fromtimestamp(mktime(entry['published_parsed']))
      link = link.url
      title = entry['title']
      article = NewsPlease.from_url(link, timeout=10)
      row=[time_of_upload,link,title,article.description,article.maintext]
      rows.append(row)
    except:
      pass

  df = pandas.DataFrame(rows, columns=['Date','URL','Title','Description','Text'])
  df[df.columns[1:]] = df[df.columns[1:]].astype(str)
  return df

### News cleaning

In [ ]:
def cleaner(df):
    data = df[['Title','Description','Text']]
    texts = data['Text'].tolist()
    
    for text in texts:
        if (len(data[data['Text']==text])>=2):
            data['Text'].replace(to_replace=text,value="None",inplace=True)
    df[['Title','Description','Text']] = data
    for column in df.columns:
        df[column] = df[column].astype(str)
    df['Date'] = pandas.to_datetime(df['Date'], format='%Y-%m-%d')
    
    df['Text'] = df['Text'].str.lower()
    for impurity in impurities:
        df['Text'] = df['Text'].str.replace(impurity,' ',regex=True)
    
    df = df.reset_index()
    return df.sort_values(by='Date', ascending=False)

### News driver

In [ ]:

companies = pandas.read_csv('/content/drive/MyDrive/Sentiment Analysis/GNews Companies URLs/BSE100.csv')
frames = []
for num in range(0,len(companies[:])):
    clear_output(wait=True)
    try:
        news_df = get_news(companies['Code'][num])
        news_df.insert(0,'GNews',('https://news.google.com/topics/'+companies['Code'][num]+'?hl=en-IN&gl=IN&ceid=IN%3Aen'))
        news_df.insert(0,'Name',companies['Name'][num])
        print((num+1),': ',news_df['Name'][0])
        print('Rows: ',len(news_df['Name']))
        if len(news_df)>=1:
            frames.append(news_df)
    except:
        pass
news = pandas.concat(frames)

100 :  Procter & Gamble Hygiene & Health Care Ltd
Rows:  1


In [ ]:
#Cleaning the news
news = cleaner(news)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


## Sentiment Analyser

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Sentiment Analysis/Model files/pytorch_model.bin",
                                                      config="/content/drive/MyDrive/Sentiment Analysis/Model files/config.json",
                                                      num_labels=3)
    label_list=['positive','negative','neutral']
analyser = SentimentIntensityAnalyzer()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##Running finBERT

In [ ]:
def finbert_model(input_df, article_part="Text"):
  ml_polarity = []
  ml_sent = []
  vader_polarity = []
  # doing the analysis for the text column
  for i in input_df[article_part]:
    inputs = tokenizer(i, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
    prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
    polarity_score = prob_0-prob_1
    vader = analyser.polarity_scores(i)
    if (i==str(numpy.nan)):
      ml_sent.append('neutral')
      ml_polarity.append(0)
      vader_polarity.append(0)
    else:
      ml_sent.append(label_list[torch.argmax(outputs[0])])
      ml_polarity.append(polarity_score)
      vader_polarity.append(vader['compound'])
  input_df[article_part+'Polarity'] = ml_polarity
  input_df[article_part+'Sentiment'] = ml_sent
  return input_df

In [ ]:
#Running model on text
news = finbert_model(news, article_part="Text")


In [ ]:
#Running model on description 
news = finbert_model(news, article_part="Description")

In [ ]:
def most_extreme(input_df):
    extreme_polarities = []
    extreme_sentences = []
    input_df.dropna()
    for text in input_df["Text"]:
        extreme_polarity = 0
        extreme_sentence = "None"

        if len(text)>10:
            sentences = sent_tokenize(text)
            
            for sentence in sentences:
                input = tokenizer(sentence, return_tensors="pt", truncation=True)
                outputs = model(**input)
                prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
                prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
                ml_sentiment = label_list[torch.argmax(outputs[0])]
                polarity_score = prob_0-prob_1
                if abs(polarity_score)>extreme_polarity:
                    extreme_polarity = polarity_score
                    extreme_sentence = sentence
        extreme_polarities.append(extreme_polarity)
        extreme_sentences.append(extreme_sentence)

    input_df["Extreme Polarity"] = extreme_polarities
    input_df["Extreme Sentence"] = extreme_sentences
    return input_df

## Data Cleaning

In [ ]:
news.drop(columns="index", inplace=True)

In [ ]:
news.sort_values(by=['Name', 'Date'], inplace=True)

In [ ]:
news['TextPolarity'] = (news['TextPolarity']*100).round()
news['DescriptionPolarity'] = (news['DescriptionPolarity']*100).round()

In [ ]:
news['TextPolarity'].where(news['Text']!="none", news['DescriptionPolarity'], inplace=True)

## Outputting to Google Sheets via gspread

In [ ]:
!pip install gspread-dataframe

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from gspread_pandas import Spread, Client
from datetime import datetime as dt
import gspread_dataframe as gd

In [ ]:
scope = ['https://www.googleapis.com/auth/spreadsheets', 
'https://www.googleapis.com/auth/drive.file',
'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Sentiment Analysis/Output CSVs/Google Sheet GSpread API stuff/credentials.json',
                                                         scope)
client = gspread.authorize(creds)


In [ ]:
config = gspread_pandas.conf.get_config(conf_dir="/content/drive/MyDrive/Sentiment Analysis/Output CSVs/Google Sheet GSpread API stuff",
                               file_name='credentials.json')

In [ ]:
s = Spread("BSE100", config = config, sheet="nimf-baf")

In [ ]:
s.sheets

[<Worksheet 'test' id:0>,
 <Worksheet '12-Jan-2022' id:173301897>,
 <Worksheet '2022-01-15old' id:147853641>,
 <Worksheet '2022-01-25' id:491576836>,
 <Worksheet '2022-02-02old' id:1667593105>,
 <Worksheet '2022-02-18' id:256279983>,
 <Worksheet '2022-02-23' id:1062386497>]

In [ ]:
s.url

'https://docs.google.com/spreadsheets/d/1epQJw2jwtDCpiL_ss6t_RVYVLj2piy6NZ8vpBLE7L4s'

In [ ]:
news.shape

(172, 11)

In [ ]:
start_cell = "A"+str(s.get_sheet_dims()[0]+1)

In [ ]:
existing_data_from_gsheet = s.sheet_to_df(header_rows=1)

In [ ]:
appended_df = existing_data_from_gsheet.append(news)

In [ ]:
appended_df.sort_values(by=['Name', 'Date'], inplace=True)

In [ ]:

s.df_to_sheet(df=appended_df, sheet="nimf-baf", start="A1", replace=True,
              freeze_headers=True)
